# 模型蒸馏 （学生结构简单，教师复杂 总的损失：学生自学习【与标准答案的差异】+权重*教师对学生的指导）

https://www.lanqiao.cn/courses/40981/learning/?id=2813910&compatibility=false

<mark>ERROR</mark>: 
* 
* 
*

训练模型的有可能准确率不易达标！
---
# 手打答案
```python 
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms

# 定义教师模型
class TeacherModel(nn.Module):
    def __init__(self):
        super(TeacherModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.model(x)

# 定义学生模型
class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.model = nn.Sequential(
        #TODO
        nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.model(x)


# 训练教师模型
def train_teacher(model, optimizer, criterion, train_loader):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        output = model(images.view(images.size(0), -1))
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step() 

# 训练学生模型
def train_student(teacher_model, student_model, optimizer, criterion, distillation_loss, train_loader, model_path='./student_model.pth'):
    teacher_model.eval()
    student_model.train()
    for images, labels in train_loader:
        #TODO
        optimizer.zero_grad()  # 在这里我将zero_grad 写为了no_grad 是错误的

        # 防止教师模型收到反向影响
        teacher_output = teacher_model(images.view(images.size(0), -1)).detach() #从计算图中分离出一个张量，返回一个新的张量，该张量与原始张量共享相同的数据，但不会参与反向传播计算。
        student_output = student_model(images.view(images.size(0), -1))

        loss = criterion(student_output, labels) + 0.45*distillation_loss(student_output, teacher_output)
        loss.backward() # 将backard 写错为了backend
        optimizer.step()
    
    torch.save(student_model.state_dict(),model_path)
        
        

# 验证模型准确率
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            output = model(images.view(images.size(0), -1))
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print('Accuracy: {:.2f}%'.format(accuracy))

if __name__ == '__main__':
    # 加载数据集
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    train_loader = torch.utils.data.DataLoader(torch.load('train_dataset.pth'), batch_size=32, shuffle=True)
    test_loader = torch.utils.data.DataLoader(torch.load('test_dataset.pth'), batch_size=32, shuffle=True)
    # 定义损失函数和优化器
    criterion = nn.NLLLoss()
    distillation_loss = nn.KLDivLoss()  # 添加蒸馏损失函数
    teacher_model = TeacherModel()
    student_model = StudentModel()
    optimizer = optim.Adam(student_model.parameters(), lr=0.001)

    # 训练教师模型
    train_teacher(teacher_model, optimizer, criterion, train_loader)

    # 训练学生模型
    for epoch in range(5):
        train_student(teacher_model, student_model, optimizer, criterion, distillation_loss, train_loader)
        evaluate_model(student_model, test_loader)
```